In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

from src.sales_data import SalesData

In [2]:
for filename in os.listdir('data'):
    if '.csv' in filename:
        df = pd.read_csv(f'data/{filename}')
        print(f'{filename}:\t{df.shape}')

sales_train.csv:	(2935849, 6)
shops.csv:	(60, 2)
test.csv:	(214200, 3)
item_categories.csv:	(84, 2)
items.csv:	(22170, 3)
sample_submission.csv:	(214200, 2)


## Import Data and Format Columns
Also, add year, month, and year_month columns. Then, aggregate to monthly sales.

In [5]:
sd = SalesData()
sd.set()

In [14]:
sd.monthly_agg()
sales = sd.monthly_sales

In [15]:
sales.head()

,date_block_num,year,month,year_month,shop_id,item_id,item_price,item_cnt_mth
0,0,2013,01,201301,0,1000,58.0,5.0
1,0,2013,01,201301,0,1001,58.0,2.0
2,0,2013,01,201301,0,10012,76.0,1.0
3,0,2013,01,201301,0,1002,58.0,2.0
4,0,2013,01,201301,0,1003,58.0,2.0


In [16]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1739022 entries, 0 to 1739021
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date_block_num  int64  
 1   year            int64  
 2   month           object 
 3   year_month      object 
 4   shop_id         object 
 5   item_id         object 
 6   item_price      float64
 7   item_cnt_mth    float64
dtypes: float64(2), int64(2), object(4)
memory usage: 106.1+ MB


# Baseline Models
Get a baseline score using Gradient Boosting and Random Forest Regressors. First run used GridSearch to find best parameters... probably overkill for our baseline models.

In [17]:
def ts_train_test_split(X, y, test_periods=1):
    periods = sorted(list(X['year_month'].unique()))
    train_periods = periods[:-test_periods]
    test_periods = periods[-test_periods:]
    
    train_mask = X['year_month'].isin(train_periods)
    X_train = X.loc[train_mask]
    y_train = y.loc[train_mask]
    
    test_mask = X['year_month'].isin(test_periods)
    X_test = X.loc[test_mask]
    y_test = y.loc[test_mask]
    return X_train, X_test, y_train, y_test

In [20]:
feat_cols = ['year_month', 'year', 'month', 'shop_id', 'item_id', 'item_price']
X = sales.loc[:, feat_cols].copy()
y = sales.loc[:, 'item_cnt_mth'].copy()

X_train, X_test, y_train, y_test = ts_train_test_split(X, y, test_periods=1)

In [21]:
X_train.head()

,year_month,year,month,shop_id,item_id,item_price
0,201301,2013,01,0,1000,58.0
1,201301,2013,01,0,1001,58.0
2,201301,2013,01,0,10012,76.0
3,201301,2013,01,0,1002,58.0
4,201301,2013,01,0,1003,58.0


In [25]:
for df in [X_train, X_test]:
    df.drop(['year_month', 'year', 'item_price'], axis=1, inplace=True)

/Users/u1b1700/.pyenv/versions/3.7.2/envs/kag/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Random Forest Regressor

In [ ]:
# rf = RandomForestRegressor(n_jobs=4)
# params = {'n_estimators': [100, 500, 1000], 'max_depth': [2, 3, 4]}
# gs = GridSearchCV(estimator=rf, param_grid=params, verbose=3)
# gs.fit(X_train, y_train.values.ravel())


In [ ]:
# for k in gs.cv_results_.keys():
#     print(f'{k}:\n{gs.cv_results_[k]}')

In [29]:
rf = RandomForestRegressor(n_estimators=500, 
                           criterion='mse', 
                           max_depth=2, 
                           min_samples_split=2, 
                           min_samples_leaf=1, 
                           min_weight_fraction_leaf=0.0, 
                           max_features='auto', 
                           max_leaf_nodes=None, 
                           min_impurity_decrease=0.0, 
                           min_impurity_split=None, 
                           bootstrap=True, 
                           oob_score=False, 
                           n_jobs=2, 
                           random_state=123, 
                           verbose=1, 
                           warm_start=False, 
                           ccp_alpha=0.0, 
                           max_samples=None)

rf.fit(X_train, y_train.values.ravel())

y_pred = rf.predict(X_test)

score = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Random Forest Regressor RMSE: {score}')

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   10.5s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   44.4s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  1.7min


Random Forest Regressor RMSE: 13.235326902255489


[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:  1.9min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.1s finished


### Gradient Boosting Regressor

In [ ]:
# gb = GradientBoostingRegressor(criterion='mse', n_iter_no_change=100)
# params = {'n_estimators': [100, 200, 400], 
#           'max_depth': [2, 3, 4], 
#           'learning_rate': [0.05, 0.1]}
# gs = GridSearchCV(estimator=gb, param_grid=params, n_jobs=1, verbose=3)
# gs.fit(X_train, y_train.values.ravel())


In [ ]:
# for k in gs.cv_results_.keys():
#     print(f'{k}:\n{gs.cv_results_[k]}')

In [ ]:
# best_score_idx = np.argmin(gs.cv_results_['rank_test_score'])
# gs.cv_results_['params'][best_score_idx]

In [30]:
gb = GradientBoostingRegressor(loss='ls', 
                               learning_rate=0.1, 
                               n_estimators=200, 
                               subsample=1.0, 
                               criterion='mse', 
                               min_samples_split=2, 
                               min_samples_leaf=1, 
                               min_weight_fraction_leaf=0.0, 
                               max_depth=2, 
                               min_impurity_decrease=0.0, 
                               min_impurity_split=None, 
                               init=None, 
                               random_state=None, 
                               max_features=None, 
                               alpha=0.9, 
                               verbose=1, 
                               max_leaf_nodes=None, 
                               warm_start=False, 
                               presort='deprecated', 
                               validation_fraction=0.1, 
                               n_iter_no_change=100, 
                               tol=0.0001, 
                               ccp_alpha=0.0)

gb.fit(X_train, y_train.values.ravel())
y_pred = gb.predict(X_test)

score = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Gradient Boosting Regressor RMSE: {score}')

      Iter       Train Loss   Remaining Time 
         1          53.8286            3.53m
         2          51.9387            3.45m
         3          50.4079            3.49m
         4          49.1677            3.52m
         5          48.1628            3.54m
         6          47.3479            3.51m
         7          46.6878            3.48m
         8          46.2276            3.46m
         9          45.7798            3.43m
        10          45.4166            3.40m
        20          44.3538            3.23m
        30          44.2013            2.97m
        40          44.0791            2.76m
        50          44.0441            2.56m
        60          43.9574            2.38m
        70          43.8851            2.20m
        80          43.7786            2.03m
        90          43.7529            1.85m
       100          43.6705            1.68m
       200          43.2841            0.00s
Gradient Boosting Regressor RMSE: 13.205525666865636


## Categorical info in shops, potentially
A quick glance into the translations of some of the shop_names in shops indicated that I might be able to break out some categorical info. The first word might be a city or some other location. Also, some of the words which have higher frequencies seem to point to either a shopping center, a mall, a megastore, etc.

In [ ]:
def clean_names(names_list):
    cleaned = []
    for n in names_list:
        n_cleaned = re.sub("[.,\")(!]", "", n)
        cleaned.append(n_cleaned.upper())
    return cleaned


def tokenize_it(n_list):
    return [t.split() for t in n_list]


def create_corpus(names_list):
    corpus = clean_names(names_list)
    corpus = tokenize_it(corpus)
    return [' '.join(item) for item in corpus]


def get_top_words(df, corpus, top_n):
    cvect = CountVectorizer()
    count_matrix = cvect.fit_transform(corpus)
    word_counts = np.sum(count_matrix.toarray(), axis=0)
    vocab = cvect.get_feature_names()
    count_rank = np.argsort(word_counts)[::-1]
    word_rank = np.array(vocab)[count_rank]  

    count_df = pd.DataFrame(data=count_matrix.toarray(), 
                            columns=vocab)
    
    df = pd.concat([df, count_df.loc[:, word_rank[:top_n]]], axis=1)
    return df

In [ ]:
# clean up names, get locations, then vectorize the top occurences
shops['clean_name'] = create_corpus(shops['shop_name'])
shops['loc_name'] = shops['clean_name'].apply(lambda x: x.split()[0])
shops = get_top_words(shops, shops['clean_name'], 10)

# remove top occurences if in loc_name
for col in shops.columns:
    if col.upper() in shops['loc_name'].unique():
        shops.drop(col, axis=1, inplace=True)

In [ ]:
shops.head()

In [ ]:
sales.head()

## Model Aggregrated Sales Data
Break out components of date and merge shop info. Then remove columns that don't make sense for modeling and get dummies for location.

In [ ]:
shops['shop_id'] = shops['shop_id'].astype(str)

sales['day'] = sales['date'].apply(lambda x: x.day)
sales['month'] = sales['date'].apply(lambda x: f'{x.month:02d}')
sales['year'] = sales['date'].apply(lambda x: x.year)
sales['year_month'] = sales['year'].astype(str) + sales['month'].astype(str)
sales['sales_day'] = sales['item_price'] * sales['item_cnt_day']

In [ ]:
# sales = pd.merge(sales, shops, on='shop_id', how='inner')

In [ ]:
drop_cols = ['shop_name', 'clean_name']
for col in drop_cols:
    sales.drop(col, axis=1, inplace=True)

In [ ]:
gb_cols = ['date_block_num', 'shop_id', 'item_id', 'month', 'year', 'loc_name', 
           'тц', 'трц', 'мега', 'тк', 'трк', 'молл', 'центральный']

model_input = sales.groupby(gb_cols)[['item_cnt_day']].sum().reset_index()

In [ ]:
model_input = pd.get_dummies(data=model_input, prefix='loc', prefix_sep='_', 
                             columns=['loc_name'], drop_first=True)

In [ ]:
X = model_input.drop('item_cnt_day', axis=1)
y = model_input.loc[:, 'item_cnt_day']
X_train, X_test, y_train, y_test = ts_train_test_split(X, y, test_periods=1)

In [ ]:
for df in [X_train, X_test]:
    df.drop(['date_block_num', 'year'], axis=1, inplace=True)

In [ ]:
X_train.head()

In [ ]:
rf = RandomForestRegressor(n_estimators=500, 
                           criterion='mse', 
                           max_depth=2, 
                           min_samples_split=2, 
                           min_samples_leaf=1, 
                           min_weight_fraction_leaf=0.0, 
                           max_features='auto', 
                           max_leaf_nodes=None, 
                           min_impurity_decrease=0.0, 
                           min_impurity_split=None, 
                           bootstrap=True, 
                           oob_score=False, 
                           n_jobs=2, 
                           random_state=123, 
                           verbose=0, 
                           warm_start=False, 
                           ccp_alpha=0.0, 
                           max_samples=None)

rf.fit(X_train, y_train.values.ravel())

y_pred = rf.predict(X_test)

score = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Random Forest Regressor RMSE: {score}')

In [ ]:
gb = GradientBoostingRegressor(loss='ls', 
                               learning_rate=0.1, 
                               n_estimators=200, 
                               subsample=1.0, 
                               criterion='mse', 
                               min_samples_split=2, 
                               min_samples_leaf=1, 
                               min_weight_fraction_leaf=0.0, 
                               max_depth=2, 
                               min_impurity_decrease=0.0, 
                               min_impurity_split=None, 
                               init=None, 
                               random_state=None, 
                               max_features=None, 
                               alpha=0.9, 
                               verbose=0, 
                               max_leaf_nodes=None, 
                               warm_start=False, 
                               presort='deprecated', 
                               validation_fraction=0.1, 
                               n_iter_no_change=100, 
                               tol=0.0001, 
                               ccp_alpha=0.0)

gb.fit(X_train, y_train.values.ravel())
y_pred = gb.predict(X_test)

score = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Gradient Boosting Regressor RMSE: {score}')

# Initial Insights
Decision Tree models not really doing the job we need it to here. The RMSE scores are terrible. Time Series modeling is likely the better approach. Let's see what diffferent aggregations look like. Then, maybe we can pick a particular shop-item combination to use as a template for our TS model...

## Sales overall by month

In [ ]:
sales['month'] = sales['month'].astype(int)

In [ ]:
sales.head()

In [ ]:
sales.info()

In [ ]:
sales.loc[sales['date_block_num'] == 24]['date'].unique()

In [ ]:
sales.loc[(sales['month'] > 9) & (sales['year'] == 2015)]

In [ ]:
def plot_trends(df, title_id):
    x = df['year_month']
    y1 = df['item_cnt_day']
    y2 = df['sales_day']

    fig, ax1 = plt.subplots(figsize=(16, 8))
    ax2 = ax1.twinx()
    ax1.plot(x, y1, 'g-', label='Item Count')
    ax2.plot(x, y2, 'b-', label='Sales')
    
    ax1.set_xlabel('Period')
    ax1.tick_params(axis='x', rotation=90)
    ax1.set_ylabel('Item Count')
    ax2.set_ylabel('Sales')
    fig.legend()
#     ax2.legend(loc='best')
    plt.title(f'{title_id}\nItem Count & Sales')
    plt.show()

In [ ]:
overall = sales.groupby(['year_month'])[['item_cnt_day', 'sales_day']].sum().reset_index()
overall.sort_values(['year_month'], inplace=True)
plot_trends(overall, 'Overall')

## Sales by City

In [ ]:
gb_cols = ['loc_name', 'year_month']
city_sales = sales.groupby(gb_cols)[['item_cnt_day', 'sales_day']].sum().reset_index()

for n in sorted(city_sales['loc_name'].unique()):
    print(f'********\n{n}\n********\n')
    subdf = city_sales.loc[city_sales['loc_name'] == n]
    print(subdf.shape)
    plot_trends(subdf, n)

## Sales by the location types

In [ ]:
loc_types = ['тц', 'трц', 'мега', 'тк', 'трк', 'молл', 'центральный']

for t in loc_types:
    print(f'********\n{t}\n********\n')
    subdf = sales.loc[sales[t] == 1]
    subdf = subdf.groupby(['year_month'])[['item_cnt_day', 'sales_day']].sum().reset_index()
    plot_trends(subdf, t)

## Item Categories
Most item categories appear to have a more general category description. Break this out into its own column. Then, merge categories to items.

In [ ]:
# split on hyphen
mask = cats['item_category_name'].str.contains(' -')
cats.loc[mask, 'major_category'] =  cats.loc[
    mask, 'item_category_name'].apply(lambda x: x.split(' -')[0])

# get description before parenthesis
mask1 = cats['item_category_name'].str.contains(' \(')
mask2 = cats['major_category'].isna()
cats.loc[mask1 & mask2, 'major_category'] =  cats.loc[
    mask1 & mask2, 'item_category_name'].apply(lambda x: x.split(' (')[0])

# whatever is left
mask = cats['major_category'].isna()
cats.loc[mask, 'major_category'] =  cats.loc[mask, 'item_category_name']

items = pd.merge(items, cats, on=['item_category_id'], how='inner')

In [ ]:
items.head()

## Sales by Major Categories

In [ ]:
sales = pd.merge(sales, items, on=['item_id'], how='inner')

for cat in sorted(sales['major_category'].unique()):
    print(f'**********\n{cat}\n**********')
    subdf = sales.loc[sales['major_category'] == cat]
    subdf = subdf.groupby('year_month')[['item_cnt_day', 'sales_day']].sum().reset_index()
    plot_trends(subdf, cat)
          

## Focus on Tops

In [ ]:
top_majorcat = 'Игры'
top_loc = 'МОСКВА'
top_loc_type = 'тц'
top_shop_id = 31
top_cat = 'Игры - XBOX 360'

In [ ]:
majorcat_mask = sales['major_category'] == top_majorcat
loc_mask = sales['loc_name'] == top_loc
shop_mask = sales['shop_id'] == top_shop_id
cat_mask = sales['item_category_name'] == top_cat

top_catloc = sales.loc[majorcat_mask & 
                       loc_mask & 
                       shop_mask & 
                       cat_mask &
                       (sales[top_loc_type] == 1)]
top_catloc.head()

In [ ]:
top_catloc['item_category_name'].unique()

In [ ]:
top_items = top_catloc.groupby(['item_name', 'item_id'])[['item_cnt_day', 'sales_day']].sum().reset_index()
top_items.sort_values(['item_cnt_day'], ascending=False, inplace=True)

In [ ]:
top_items.head(20)

In [ ]:
items.head()

In [ ]:
items.loc[items['item_id'] == 3342]

In [ ]:
top_catloc['item_name'].unique()

In [ ]:
top_catloc.columns

In [ ]:
for cat_name in sorted(top_catloc['item_category_name'].unique()):
    print(f'*********\n{cat_name}\n***********')
    subdf = top_catloc.loc[top_catloc['item_category_name'] == cat_name]
    subdf = subdf.groupby('year_month')[['item_cnt_day', 'sales_day']].sum().reset_index()
    plot_trends(subdf, cat_name)

In [ ]:
gb_cols = ['shop_id', 'shop_name', 'clean_name', 'loc_name',
           'тц', 'трц', 'мега', 'тк', 'трк', 'молл', 'центральный']
shop_agg = top_catloc.groupby(gb_cols)[['item_cnt_day', 'sales_day']].sum().reset_index()
shop_agg.sort_values('item_cnt_day', ascending=False, inplace=True)

In [ ]:
shop_agg.head()

In [ ]:
sales.head()

In [ ]:
sales.columns

In [ ]:
items['major_category'].value_counts()

In [ ]:
items.head()

In [ ]:
cats.loc[cats['major_category'] == 'Книги']

In [ ]:
items['item_category_id'].value_counts()

In [ ]:
cats

In [ ]:
items.loc[items['item_category_id'] == 40]

In [ ]:
items.head(20)

In [ ]:
cats.loc[40:, :]

In [ ]:
sales.head()

In [ ]:
sales.columns

# NLP on Shop Names

In [ ]:
corpus = create_corpus(shop_names)

In [ ]:
vect = TfidfVectorizer()
vect_matrix = vect.fit_transform(corpus)

In [ ]:
vect_matrix.toarray()

In [ ]:
tfidf_array = vect_matrix.toarray()
count_array = count_matrix.toarray()

In [ ]:
np.sum(count_array, axis=0)

In [ ]:
np.sum(tfidf_array, axis=0)

In [ ]:
cvect.get_feature_names()

In [ ]:
sorted(cleaned_names)

In [ ]:
shops.head(20)

In [ ]:
items.head()

In [ ]:
cats.head()